In [7]:
import pandas as pd
from datetime import date
import holidays

bar_holiday = holidays.HolidayBase()

bar_holiday.append('2019-04-19','2019-04-22','2019-05-01','2019-06-10','2019-06-24','2019-08-15','2019-09-11','2019-09-24','2019-10-12',
                  '2019-11-01','2019-12-06','2019-12-25','2019-12-26','2020-01-01','2020-01-06','2020-04-10','2020-04-13',
                  '2020-05-01','2020-06-01','2020-06-24','2020-08-15','2020-09-11','2020-09-24','2020-10-12','2020-12-08',
                  '2020-12-25','2020-12-26','2021-01-01','2021-01-06','2021-04-02','2021-04-05','2021-05-01','2021-05-24',
                  '2021-09-11','2021-09-24','2021-10-12','2021-11-01','2021-12-06','2021-12-08','2021-12-25','2022-01-01','2022-01-06',
                  '2022-04-15','2022-04-18','2022-06-06','2022-06-24','2022-08-15','2022-09-24','2022-09-26','2022-10-12','202-11-01',
                  '2022-12-06','2022-12-08','2022-12-26','2023-01-01','2023-01-06','2023-04-07','2023-04-10')

In [8]:
range_of_dates = pd.date_range('2019-03-01', '2023-12-31')

holiday_df = pd.DataFrame(
    index=range_of_dates, 
    data={'is_holiday': [date in bar_holiday for date in range_of_dates]}
)

In [9]:
holiday_df

,is_holiday
2019-03-01,False
2019-03-02,False
2019-03-03,False
2019-03-04,False
2019-03-05,False
...,...
2023-12-27,False
2023-12-28,False
2023-12-29,False
2023-12-30,False


In [27]:
def create_calendar(df):
        
    weekend = {1: False, 2: False, 3: False, 4: False, 5: False, 6: True, 7: True}
    
    def get_season(month):
        x = month % 12 // 3 + 1
        if x == 1:
            season = 'winter'
        elif x == 2:
            season = 'spring'
        elif x == 3:
            season = 'summer'
        elif x == 4:
            season = 'autumn'
        else:
            season = 'unknown'
        return season

    return (df
            .assign(year=df.index.year.astype('uint16'), month=df.index.month.astype('uint8'), 
                    day=df.index.day.astype('uint8'), weekday=df.index.isocalendar().day.astype('uint8'),
                    weekend=lambda df_: df_['weekday'].map(weekend), season=lambda df_: df_['month'].apply(get_season).astype('category'),)
    )

In [24]:
create_calendar(holiday_df)

,is_holiday,year,month,day,weekday,weekend,season
2019-03-01,False,2019,3,1,5,False,spring
2019-03-02,False,2019,3,2,6,True,spring
2019-03-03,False,2019,3,3,7,True,spring
2019-03-04,False,2019,3,4,1,False,spring
2019-03-05,False,2019,3,5,2,False,spring
...,...,...,...,...,...,...,...
2023-12-27,False,2023,12,27,3,False,winter
2023-12-28,False,2023,12,28,4,False,winter
2023-12-29,False,2023,12,29,5,False,winter
2023-12-30,False,2023,12,30,6,True,winter


In [25]:
create_calendar(holiday_df).dtypes

is_holiday        bool
year            uint16
month            uint8
day              uint8
weekday          uint8
weekend           bool
season        category
dtype: object

In [26]:
create_calendar(holiday_df).to_parquet(fr'..\data\processed\calendar.parquet')